# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
num_new_rows = 0
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    num_new_rows = sum(1 for line in f)
    # deduct one for the header row
    num_new_rows = num_new_rows - 1
    print(num_new_rows)

6820


# Part II. Complete the Apache Cassandra coding portion of your project. 

Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Initialize database connection

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create tables

**Print out column number and name for convenience**

In [8]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    cols = next(csvreader)
    for i, t in enumerate(cols):
        print(f'{i}: {t}')

0: artist
1: firstName
2: gender
3: itemInSession
4: lastName
5: length
6: level
7: location
8: sessionId
9: song
10: userId


### Table 1 - song_played
#### Question to answer
Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

#### Data model
* sessionId *int* **primary key**
* itemInSession *int* **clustering column**
* artist *text*
* song *text*
* length *float*

We use sessionId as primary key as that is our main filtering option. In addition, itemInSession is used as a clustering column. This enforces queries to use both sessionId and itemInSession as inputs and the remaining columns in the table are the data that we want to get out of the query.

In [9]:
query = """
CREATE TABLE IF NOT EXISTS song_played (
    sessionId int,
    itemInSession int,
    artist text,
    song text,
    length float,
    PRIMARY KEY (sessionId, itemInSession)
)
"""

try:
    session.execute(query)
except Exception as e:
    print(e)

### Table 2 - songs_in_session
#### Question to answer
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Data model
* userId *int* **partition key**
* sessionId *int* **partition key**
* itemInSession *int* **clustering column**
* artist *text*
* song *text*
* firstName *text*
* lastName *text*

We use userId and sessionId as composite partition key to identify a specific sesseion for a user. itemInSession is used as a clustering column in order to sort the data in the order that the songs were played in the session. The remaining columns in the table are the items we want to use as output for the query.

In [10]:
query = """
CREATE TABLE IF NOT EXISTS songs_in_session (
    userId int,
    sessionId int,
    itemInSession int,
    artist text,
    song text,
    firstName text,
    lastName text,
    PRIMARY KEY ((userId, sessionId), itemInSession)
)
"""

try:
    session.execute(query)
except Exception as e:
    print(e)

### Table 3 - songs_listened_by
#### Question to answer
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Data model
* song *text* **primary key**
* userId *int* **clustering column**
* firstName *text*
* lastName *text*

We must use song name as primary key as it is the input for out query. In addition, we need to define userId as a clustering column to be able to get each individual listener for a song. The remaining firstName and lastName columns in the table are the items that we want to know.

In [11]:
query = """
CREATE TABLE IF NOT EXISTS songs_listened_by (
    song text,
    userId int,
    firstName text,
    lastName text,
    PRIMARY KEY (song, userId)
)
"""

try:
    session.execute(query)
except Exception as e:
    print(e)

## Insert data

In [12]:
def read_item_count(table, primary_key):
    """ Function that reads the total amount of rows in a table.
    
    Returns amount of items in table upon success. Otherwise returns -1.
    """
    query = f'SELECT COUNT({primary_key}) FROM {table}'
    try:
        results = session.execute(query)
        return results.current_rows[0][0] # first row, first value
    except Exception as e:
        print(e)
        return -1

In [13]:
table1_item_count_before = read_item_count('song_played', 'sessionId')
table2_item_count_before = read_item_count('songs_in_session', 'userId')
table3_item_count_before = read_item_count('songs_listened_by', 'song')

In [14]:
file = 'event_datafile_new.csv'

# initialize empty lists to keep track of INSERT failures
table1_insertfailures = []
table2_insertfailures = []
table3_insertfailures = []

# iterate through input file and make all writes to the tables in the same loop
# to avoid iterating the same input file several times.
# catch any problemsn upon failure and add the data to failure lists for book-keeping
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try:
            # TABLE 1
            query = "INSERT INTO song_played (sessionId, itemInSession, artist, song, length)"
            query = query + " VALUES (%s, %s, %s, %s, %s)"
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            table1_insertfailures.extend(line)
            
        try:
            # TABLE 2
            query = "INSERT INTO songs_in_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
            query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            table2_insertfailures.extend(line)
            
        try:
            # TABLE 3
            query = "INSERT INTO songs_listened_by (song, userId, firstName, lastName)"
            query = query + " VALUES (%s, %s, %s, %s)"
            session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            table3_insertfailures.extend(line)            

#### Verify that data has been inserted to the table
We've read the amount of rows in the table before and after the INSERT operation.
In case the amount of rows is the same or if the lists tha contain information about failed INSERTS contain data we know that something went wrong.

These checks are redundant from the point of view that we want to identify failures during operation but I decided to keep both since the difference in item amounts give us information of how many rows were written. The lists of failed INSERST on the other hand could give us valuable information to debug any issues that arise.

In [15]:
table1_item_count_after = read_item_count('song_played', 'sessionId')
table2_item_count_after = read_item_count('songs_in_session', 'userId')
table3_item_count_after = read_item_count('songs_listened_by', 'song')

In [16]:
print(f"Amount of rows in input file: {num_new_rows}")

if table1_item_count_after == table1_item_count_before or len(table1_insertfailures):
    print("There were problems inserting data into table 1")
else:
    print(f"Inserted/updated {table1_item_count_after-table1_item_count_before} rows to table 1.")

if table2_item_count_after == table2_item_count_before or len(table2_insertfailures):
    print("There were problems inserting data into table 2")
else:
    print(f"Inserted/updated {table2_item_count_after-table2_item_count_before} rows to table 2.")
    
if table3_item_count_after == table3_item_count_before or len(table3_insertfailures):
    print("There were problems inserting data into table 3")
else:
    print(f"Inserted/updated {table3_item_count_after-table3_item_count_before} rows to table 3.")

Amount of rows in input file: 6820
Inserted/updated 6820 rows to table 1.
Inserted/updated 6820 rows to table 2.
Inserted/updated 6618 rows to table 3.


## Create queries to ask the following three questions of the data

In [17]:
def run_query(query, params):
    """Separate function for running a query in order to avoid
    having to use the same try cathces everywhere.
    """
    try:
        return session.execute(query, params)
    except Exception as e:
        print(e)
        return None

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [18]:
query = """
SELECT artist, song, length 
FROM song_played
WHERE sessionId = %s AND itemInSession = %s
"""
results = run_query(query, (338,4))
print(results[0])

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [19]:
query = """
SELECT artist, song, firstName, lastName
FROM songs_in_session
WHERE userId = %s AND sessionId = %s
"""
results = run_query(query, (10,182))
if results:
    for item in results:
        print(item)      

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [20]:
query = """
SELECT firstName, lastName
FROM songs_listened_by
WHERE song = %s
"""
results = run_query(query, ['All Hands Against His Own'])
if results:
    for item in results:
        print(item) 

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [21]:
try: 
    session.execute("DROP TABLE song_played")
except Exception as e: 
    print (e)
try: 
    session.execute("DROP TABLE songs_in_session")
except Exception as e: 
    print (e)
try: 
    session.execute("DROP TABLE songs_listened_by")
except Exception as e:
    print (e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()